In [ ]:
# default_exp model

# Model

In [ ]:
#hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#export
import torch
import torch.nn as nn
from fastai.basics import *
from fastai.vision.all import *

In [ ]:
#export

class BaselineMTM(Module):
    def __init__(self, arch, n_distortion, n_sev, pretrained=True):
        store_attr()
        self.encoder = TimeDistributed(create_body(arch, pretrained=pretrained))
        n_features = dummy_eval(self.encoder.module, (300, 300)).shape[1]
        self.head = TimeDistributed(create_head(n_features, n_distortion + n_sev))
    
    def forward(self, x):
        feature_map = self.encoder(torch.stack(x, dim=1))
        out = self.head(feature_map).mean(dim=1)
        return [out[:, :self.n_distortion], out[:, self.n_distortion:]]
   
    @staticmethod
    def splitter(model): 
        return [params(model.encoder), params(model.head)]

In [ ]:
#export

class MultiScaleBackbone(Module):
    def __init__(self, arch, pretrained=True):
        store_attr()
        self.backbone = create_body(arch, pretrained=pretrained)
        self.hooks = hook_outputs(list(self.backbone.children())[4:-1], detach=False)
    
    def forward(self, x):
        feat_map_last = self.backbone(x)
        feat_maps = self.hooks.stored
        return torch.cat([AdaptiveConcatPool2d()(fm) for fm in [*feat_maps, feat_map_last]], dim=1)
    
class MultiScaleMTM(Module):
    def __init__(self, arch, n_distortion, n_sev, pretrained=True):
        store_attr()
        self.encoder = TimeDistributed(MultiScaleBackbone(arch, pretrained=pretrained))
        n_features = dummy_eval(self.encoder.module, (300, 300)).shape[1]
        self.head = TimeDistributed(create_head(n_features, n_distortion + n_sev))
    
    def forward(self, x):
        x = torch.stack(x, dim=1)
        feature_map = self.encoder(x)
        out = self.head(feature_map).mean(dim=1)
        return [out[:, :self.n_distortion], out[:, self.n_distortion:]]
   
    @staticmethod
    def splitter(model): 
        return [params(model.encoder), params(model.head)]

In [ ]:
#export

class SeqMTM(Module):
    def __init__(self, arch, n_distortion, n_sev, num_rnn_layers=1, pretrained=True):
        store_attr()
        self.encoder = TimeDistributed(nn.Sequential(
            create_body(arch, pretrained=pretrained), 
            nn.AdaptiveAvgPool2d(1), 
            Flatten()
        ))
        n_features = dummy_eval(self.encoder.module, (300, 300)).shape[1]
        self.rnn = nn.LSTM(n_features, n_features, num_layers=num_rnn_layers, batch_first=True)
        self.head = LinBnDrop(num_rnn_layers * n_features, n_distortion + n_sev)
    
    def forward(self, x):
        x = self.encoder(torch.stack(x, dim=1))
        bs = x.shape[0]
        _, (h, _) = self.rnn(x)
        out = self.head(h.view(bs,-1))
        return out[:, :self.n_distortion], out[:, self.n_distortion:]
    
    @staticmethod
    def splitter(model):
        return [params(model.encoder), params(model.rnn) + params(model.head)]